In [1]:
import os
import sys
os.environ["PYSPARK_PYTHON"]='/opt/anaconda/envs/bd9/bin/python'
os.environ["SPARK_HOME"]='/usr/hdp/current/spark2-client'
os.environ["PYSPARK_SUBMIT_ARGS"]='--num-executors 4 pyspark-shell'

spark_home = os.environ.get('SPARK_HOME', None)
if not spark_home:
    raise ValueError('SPARK_HOME environment variable is not set')

sys.path.insert(0, os.path.join(spark_home, 'python'))
sys.path.insert(0, os.path.join(spark_home, 'python/lib/py4j-0.10.7-src.zip'))
exec(open(os.path.join(spark_home, 'python/pyspark/shell.py')).read())

Welcome to
      ____              __
     / __/__  ___ _____/ /__
    _\ \/ _ \/ _ `/ __/  '_/
   /__ / .__/\_,_/_/ /_/\_\   version 2.4.7
      /_/

Using Python version 3.6.5 (default, Apr 29 2018 16:14:56)
SparkSession available as 'spark'.


In [2]:
from pyspark import SparkConf
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.sql.types import *
from pyspark import Row
import json

conf = SparkConf()

spark = (SparkSession
         .builder
         .config(conf=conf)
         .appName("rez_app")
         .getOrCreate())

In [ ]:
#spark

In [ ]:
#!hdfs dfs -ls /labs/laba01/ml-100k

In [3]:
dataAll = sc.textFile("/labs/laba01/ml-100k/u.data").map(lambda x: x.split("\t")).repartition(8)
data328 = sc.textFile("/labs/laba01/ml-100k/u.data").map(lambda x: x.split("\t")).filter(lambda x: x[1] == "328").repartition(8)
item328 = sc.textFile("/labs/laba01/ml-100k/u.item").map(lambda x: x.split("|")).filter(lambda x: x[0] == "328").repartition(8)

In [4]:
rate_328 = (dataAll.filter(lambda x: x[1] == "328")
 .map(lambda x: (x[2], len(x[2])))
 .reduceByKey(lambda x, y: x + y)
 .sortBy(lambda x: x[0])
 .map(lambda x: x[1]))

In [5]:
rate_all = (dataAll
 .map(lambda x: (x[2], len(x[2])))
 .reduceByKey(lambda x, y: x + y)
 .sortBy(lambda x: x[0])
 .map(lambda x: x[1]))

In [6]:
df_json = {'hist_film':rate_328.collect(),
      'hist_all':rate_all.collect()}

In [7]:
import json
with open('lab01.json', 'w') as f:
    json.dump(df_json, f)

In [8]:
spark.stop()